# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770417268534                   -0.53    9.0         
  2   -2.771688103569       -2.90       -1.31    1.0    164ms
  3   -2.771714497930       -4.58       -2.54    1.0    214ms
  4   -2.771714683416       -6.73       -3.47    1.0    240ms
  5   -2.771714712633       -7.53       -3.84    2.0    196ms
  6   -2.771714715171       -8.60       -5.00    1.0    171ms
  7   -2.771714715245      -10.13       -5.21    2.0    213ms
  8   -2.771714715250      -11.35       -6.34    1.0    179ms
  9   -2.771714715250      -12.86       -6.62    2.0    210ms
 10   -2.771714715250      -14.21       -7.20    1.0    430ms
 11   -2.771714715250      -14.88       -8.00    2.0    291ms
 12   -2.771714715250   +  -14.57       -9.27    1.0    710ms


-0.00013457350130034038

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770409816184                   -0.53    9.0         
  2   -2.771771726674       -2.87       -1.31    1.0    166ms
  3   -2.771801651629       -4.52       -2.56    1.0    195ms
  4   -2.771802030123       -6.42       -3.49    1.0    170ms
  5   -2.771802073232       -7.37       -3.97    2.0    217ms
  6   -2.771802074410       -8.93       -4.64    1.0    189ms
  7   -2.771802074470      -10.22       -5.36    1.0    177ms
  8   -2.771802074476      -11.25       -5.82    2.0    214ms
  9   -2.771802074476      -12.74       -6.48    1.0    184ms
 10   -2.771802074476      -13.57       -6.90    1.0    204ms
 11   -2.771802074476   +  -14.05       -7.44    2.0    227ms
 12   -2.771802074476      -14.01       -8.32    1.0    203ms


0.017612220009871747

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457350130034038
Displaced dipole:  0.017612220009871747
Polarizability :   1.7746793511172088


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920855136e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551606759e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852767666337e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694602976285e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088788876326e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.293785461264e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 1.263375516026e-12
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 1.263375516026e-12
[ Info: GMRES linsolve in iter 2; step 1: normres = 7.477113757658e-11
[ Info: GMRES linsolve in iter 2; step 2: normres = 1.392450505962e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 2.698303259901e-13
[ Info: GMRES linsolve in iter 2; finished at step 3: normres = 2.698303259901e-13
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ *  norm of residual = 2.698119697970805e-13
└ *  number of operat

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.